In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import random
import pickle
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,mean_absolute_error
import matplotlib.pyplot as plt
import gc

In [ ]:
def get_x(data):
    x=torch.stack([t[0] for t in data])
    return x
def get_y(data):
    y=torch.stack([torch.tensor(t[1]) for t in data])
    return y
with open("..//data//train_data.pkl",'rb') as f:
    train_data=pickle.load(f)

x_train,y_train=get_x(train_data),get_y(train_data)
y_train_classification,y_train_localization=y_train[:,0].to(torch.long),y_train[:,1:]
del train_data
with open("..//data//val_data.pkl",'rb') as f:
    val_data=pickle.load(f)
x_val,y_val=get_x(val_data),get_y(val_data)
y_val_classification,y_val_localization=y_val[:,0].to(torch.long),y_val[:,1:]
del val_data

# print(x_val)

In [4]:
# Build a skip connection network

device='cuda'
class Block(nn.Module):
    expansion=4
    def __init__(self,in_channels,out_channels,downsample=None,stride=1):
        super(Block,self).__init__()
        self.conv1=nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=1,padding=0,bias=False)
        self.bn1=nn.BatchNorm2d(out_channels)
        self.conv2=nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=stride,padding=1,bias=False)
        self.bn2=nn.BatchNorm2d(out_channels)
        self.conv3=nn.Conv2d(out_channels,out_channels*Block.expansion,kernel_size=1,stride=1,padding=0,bias=False)
        self.bn3=nn.BatchNorm2d(out_channels*Block.expansion)
        self.downsample=downsample
    def forward(self,x):
        identity=x
        out=self.conv1(x)
        out=self.bn1(out)
        out=F.relu(out)
        out=self.conv2(out)
        out=self.bn2(out)
        out=F.relu(out)
        out=self.conv3(out)
        out=self.bn3(out)
        if self.downsample is not None:
            identity=self.downsample(identity)
        out+=identity
        out=F.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self,layers,output_size):
        super(ResNet,self).__init__()
        self.in_channels=64
        self.conv1=nn.Conv2d(1,self.in_channels,kernel_size=7,stride=2,padding=3,bias=False)
        self.bn1= nn.BatchNorm2d(self.in_channels)
        self.maxpool1=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)   

        #Define Block layers
        self.layer1=self._make_layer(layers[0],64,1)
        self.layer2=self._make_layer(layers[1],128,2)
        self.layer3=self._make_layer(layers[2],256,2)
        self.layer4=self._make_layer(layers[3],512,2)

        self.avgpool=nn.AdaptiveAvgPool2d((1,1))

        self.fc=nn.Linear(64*Block.expansion,output_size)

    def _make_layer(self,num_blocks,out_channels,stride=1):
        downsample=None
        if stride!=1 or self.in_channels!=out_channels*Block.expansion:
            downsample=nn.Sequential(nn.Conv2d(self.in_channels,out_channels*Block.expansion,kernel_size=1,stride=stride,padding=0,bias=False),
                                     nn.BatchNorm2d(out_channels*Block.expansion))
        layers=[]
        layers.append(Block(in_channels=self.in_channels,out_channels=out_channels,downsample=downsample,stride=stride))
        self.in_channels=out_channels*Block.expansion
        for _ in range(num_blocks-1):
            layers.append(Block(in_channels=self.in_channels,out_channels=out_channels))
        return nn.Sequential(*layers)
    def forward(self, inputs):
        inputs=inputs.to(device)
        out=self.conv1(inputs)
        
        out=self.bn1(out)
        out=self.maxpool1(out)
        out=self.layer1(out)
        out=self.layer2(out)
        out=self.layer3(out)
        out=self.layer4(out)
        out=self.avgpool(out)
        out=torch.flatten(out,1)
        out=self.fc(out)
        return out

layers=[3,4,6,3]
output_size=3
model=ResNet(layers,output_size).to(device)


In [ ]:
def get_predictions(x):
    BATCH_SIZE=100
    classification_predictions=torch.tensor([])
    localization_predictions=torch.tensor([])
    model.eval()
    for i in range(0,len(x),BATCH_SIZE):
        y_hat=model(x[i:i+BATCH_SIZE]).to('cpu')
        y_hat_classification=F.softmax(y_hat[:,0])
        y_hat_localization=y_hat[:,1:]
        _,pred=torch.max(y_hat_classification,1)
        classification_predictions=torch.cat((classification_predictions,pred),dim=0)
        localization_predictions=torch.cat((localization_predictions,y_hat_localization),dim=0)
        torch.cuda.empty_cache()
    model.train()
    return classification_predictions.to(torch.long),localization_predictions


In [ ]:
EPOCHS=10
MINI_BATCH_SIZE=32
accumalation_step=1
loss_classification_fn=nn.CrossEntropyLoss()
loss_localization_fn=nn.MSELoss()
optimizer=torch.optim.adam(model.parameters(),lr=1e-2)


train_loss_progress=[]
train_loss_hist=[]
val_loss_hist=[]

training_accuracy_hist=[]
val_accuracy_hist=[]

training_mae_hist=[]
val_mae_hist=[]
for epoch in range(EPOCHS):
    model.eval()

    classification_predictions,localization_predictions=get_predictions(x_val)
    val_acc=accuracy_score(y_val.numpy(), classification_predictions.numpy())
    val_mae=mean_absolute_error(y_val.numpy(),localization_predictions.numpy())
    val_mae_hist.append(val_mae)
    val_accuracy_hist.append(val_acc)


    classification_predictions,localization_predictions=get_predictions(x_train)
    training_acc=accuracy_score(y_train.numpy(), classification_predictions.numpy())
    training_mae=mean_absolute_error(y_train.numpy(),localization_predictions.numpy())
    training_mae_hist.append(training_mae)
    training_accuracy_hist.append(training_acc)
    print(training_acc,val_acc)
    print(training_mae,val_mae)
    model.train()
    for i in range(0,len(x_train),MINI_BATCH_SIZE):
        y_hat=model(x_train[i:i+MINI_BATCH_SIZE]).to('cpu')
        classification_loss=loss_classification_fn(y_hat[:,0],y_train_classification[i:i+MINI_BATCH_SIZE])
        localization_loss=loss_localization_fn(y_hat[:,1:],y_train_localization[i:i+MINI_BATCH_SIZE])
        loss=loss/accumalation_step
        loss.backward()
        if (i+1)% accumalation_step==0:
            optimizer.step()
            optimizer.zero_grad()
            torch.cuda.empty_cache()
    torch.cuda.empty_cache()

In [ ]:
# plt.plot(train_loss_hist, label='List 1',color='blue')
# plt.plot(val_loss_hist, label='List 2',color='red')
# plt.show()
plt.plot(training_accuracy_hist[0:], label='List 1',color='blue')
plt.plot(val_accuracy_hist[0:], label='List 2',color='red')
plt.show()

classification_predictions,localization_predictions=get_predictions(x_val)
report = classification_report(y_val.numpy(), classification_predictions.numpy(), target_names=[str(i) for i in range(200)])
print(report)